### Reconstruct momena (4-vectors) from Mandelstam-like invariants

In [10]:
import numpy as np
from scipy.optimize import root

In [11]:
def mdot(p, q):
    """" mostely negative """
    return p[0]*q[0] - np.dot(p[1:], q[1:])

def boost(v, beta):
    """Lorentz boost of 4-vector v by 3-velocity beta"""
    b2 = np.dot(beta, beta)
    if b2 >= 1.0:
        raise ValueError("Velocity must be less than the speed of light")
    if b2 == 0.0:
        raise ValueError("Velocity must be non-zero")
    g = 1.0 / np.sqrt(1.0 - b2)

    id3 = np.identity(3)
    bb_norm = np.outer(beta, beta) / b2

    l = np.zeros((4, 4))
    l[0, 0] = g
    l[0, 1:] = g * beta
    l[1:, 0] = g * beta
    l[1:, 1:] = id3 + bb_norm * (g - 1)

    return np.dot(l, v)

In [12]:
def build_2to3_momenta(d12, md23, d34, d45, md15, mt_2, phi0=0.1, theta0=0.2):
    """
    Reconstruct 2->3 kinematics (two massive, three massless).
    All momenta outgoing: k0,k1,k2,k3,k4.
    """
    # 1 and 2 incoming massive
    s = 2*d12 + 2*mt_2

    if s <= 4*mt_2:
        raise ValueError("Invariant mass too small for top pair production")

    # (p3+p4+p5)^2 = s
    d35 = s/2 - d34 - d45
    
    # CoM frame
    E1 = np.sqrt(s)/2
    p1 = np.array([E1, 0, 0, np.sqrt(E1**2 - mt_2)])
    p2 = np.array([E1, 0, 0, -np.sqrt(E1**2 - mt_2)])

    # treat 3 and 4 (massless outgoing) as a cluster of mass M34
    M34_2 = 2*d34
    if M34_2 < 0:
        raise ValueError("Invariant mass squared of cluster 34 is negative")

    # in CoM frame 5 and 34 are back-to-back
    E5 = (s - M34_2)/(2*np.sqrt(s))
    if E5 < 0:
        raise ValueError("Invariant mass too large for given s")
    p5L = E1*E5 - d15
    if p5L**2 > E5**2:
        raise ValueError("No real solution for p5L")
    p5T = np.sqrt(E5**2 - p5L**2)

    p5 = np.array([E5, p5T, 0.0, p5L]) # choose eventplane y=0

    # Clustern 34 in its restframe (3 and 4 outgoing massless/back-to-back in cluster restframe)
    M34 = np.sqrt(M34_2)
    E = M34 / 2

    Q = p1 + p2 - p5
    beta = Q[1:] / Q[0]
    beta_2 = np.dot(beta, beta)
    
    # solve for angles of p3 in 34 restframe
    def equations(x):
        theta, phi = x
        p3_star = np.array([E,
                            E*np.sin(theta)*np.cos(phi),
                            E*np.sin(theta)*np.sin(phi),
                            E*np.cos(theta)])
        p3_com = boost(p3_star, beta)
        inv35 = mdot(p3_com, p5)
        inv23 = mdot(p2, p3_com)
        return [inv35 - d35, inv23 - d23]

    sol = root(equations, [theta0, phi0])
    if not sol.success:
        raise RuntimeError("Angle solve did not converge: " + sol.message)
    theta_, phi_ = sol.x

    # p3 and p4 in 34 restframe
    p3_ = np.array([E,
                        k*np.sin(theta_)*np.cos(phi_),
                        k*np.sin(theta_)*np.sin(phi_),
                        k*np.cos(theta_)])
    p3 = boost(p3_star, beta)
    p4 = Q - p3

    # 8. Convert to all–outgoing momenta
    k1 = -p1
    k2 = -p2
    k3 = p3
    k4 = p4
    k5 = p5

    return k1, k2, k3, k4, k5

In [13]:
# invariants from example 
d12 = -11/7
d23 = -7/5
d34 = -5/27
d45 = -17/5
d15 = -11/17

mt2 = 1.

# They are purely mathematical: do not match physical momenta

In [14]:
# p1, p2, p3, p4, p5 = build_2to3_momenta(-d12, d23, -d34, -d45, d15, mt2, phi0=0.4, theta0=0.6)

In [15]:
def mom_to_inv(p1, p2, p3, p4, p5):
    d12 = mdot(p1, p2)
    d23 = mdot(p2, p3)
    d34 = mdot(p3, p4)
    d45 = mdot(p4, p5)
    d15 = mdot(p1, p5)
    mt2 = mdot(p1, p1)

    mt2_check = mdot(p2, p2)
    if abs(mt2 - mt2_check) > 0.000000001:
        raise ValueError(f"Masses of p1 and p2 do not match: {mt2} vs {mt2_check}")
    return d12, d23, d34, d45, d15, mt2

In [ ]:
p3 = -np.array([500, 0, 0, 500])
p4 = -np.array([500, 0, 0, -500])
p5 = np.array([367.16537041325221935, 208.12187454434831579, 131.13753674354686041, 272.57777059709434297])
p1 = np.array([451.16038194215792601, -254.95547824215555011, -127.96321306901207038, -303.88644976300315648])
p2 = np.array([181.67424764458991149, 46.833603697807077992, -3.1743236745347509498, 31.308679165908877451])

print('p1 + p2 + p3 + p4 + p5 = ', p1 + p2 + p3 + p4 + p5)
d12, d23, d34, d45, d15, mt2 = mom_to_inv(p1, p2, p3, p4, p5)
print('Invariants are given by (0->ttqqg convention):', d12, d23, d34, d45, d15, mt2)
print('masses:', np.sqrt(mdot(p1, p1)), np.sqrt(mdot(p2, p2)), np.sqrt(mdot(p3, p3)), np.sqrt(mdot(p4, p4)), mdot(p5, p5))

p1 + p2 + p3 + p4 + p5 =  [ 5.68434189e-14 -1.70530257e-13  2.84217094e-14  5.68434189e-14]
Invariants are given by (0->ttqqg convention): 103012.7934867478 -75182.78423934052 500000 -319871.5705051733 318325.75235541014 29821.836099999928
masses: 172.6899999999998 172.69 0.0 0.0 -5.820766091346741e-11


In [17]:
# normalization of invariants according to paper

s34 = mdot(p3 + p4, p3 + p4)

print('s34 invariant check (should be 2*d34):', s34, 2*d34)

d12_ = d12/s34
d23_ = d23/s34
d34_ = d34/s34
d45_ = d45/s34
d15_ = d15/s34
mt2_ = mt2/s34

print('Normalized invariants are given by (0->ttqqg convention):', d12_, d23_, d34_, d45_, d15_, mt2_)

s34 invariant check (should be 2*d34): 1000000 1000000
Normalized invariants are given by (0->ttqqg convention): 0.1030127934867478 -0.07518278423934052 0.5 -0.3198715705051733 0.3183257523554101 0.02982183609999993
